In [ ]:
import sys
import time
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import geopandas as gpd 

sys.path.append("../")
from config.config import GEOSPHERE_INCA_TS_URL
from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path, calculate_date_of_interest_x_hours_before

In [3]:
# Load paths from the YAML file
paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

In [4]:
def get_geosphere_data(parameters: list = ["T2M", "RR"],
                       start: str = '2021-08-01T00:00', 
                       end: str = '2021-08-01T00:00', 
                       lat_lon: list = ['48.206248, 16.367569'], 
                       output_format: str = "geojson"):
    """
    function for retrieving data from geosphere API
    """
    
    lat_lon_params = '&lat_lon='.join(lat_lon)

    parameter_string_for_url = ""
    for a in parameters:
        parameter_string_for_url += f"parameters={a}&"
    
    url = f'{GEOSPHERE_INCA_TS_URL}?{parameter_string_for_url[:-1]}&start={start}&end={end}&lat_lon={lat_lon_params}&output_format={output_format}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Request failed with status code {response.status_code}.')

def get_lat_lon(row: pd.Series):
    point = (row["geometry"].y, row["geometry"].x)
    return point

def get_time_range(row: pd.Series, days: int):
    end = row.date
    start = end - timedelta(days=days)
    return start.isoformat(), end.isoformat()

def transform_date(input_date):
    """transforms date from "%m/%d/%Y" into '%Y-%m-%dT12:00' format"""
    date_object = datetime.strptime(input_date, "%m/%d/%Y")
    output_date = date_object.strftime('%Y-%m-%dT12:00')
    return str(output_date)


In [5]:
# loading fire dataset
event_data = gpd.read_file(paths["fire_events"]["final"])
event_data["date"] = event_data["date"].apply(transform_date)
event_data["year"] = event_data["year"].astype("int")
event_data_subset = event_data[event_data["year"] >= 2012]
event_data_subset.to_crs("EPSG:4326", inplace=True)

c:\Users\David\anaconda3\envs\pymc_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
with open(paths["inca"]["training_data"], 'a') as f:
    params = ["T2M", "RR", "UU", "VV", "RH2M"]

    for i, row in event_data_subset.loc[690:].iterrows():
        transformed_point = get_lat_lon(row)
        lat_lon = [f"{transformed_point[0]}, {transformed_point[1]}"]
        start_date = calculate_date_of_interest_x_hours_before(row.date, hours=24)
        data = get_geosphere_data(params, start_date, row.date, lat_lon)
        f.write(f'{i}')
        json.dump(data, f)
        f.write('\n')
        print(i)
f.close()

690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
1219
1222
1225
1231
1233
1234
1236
1237
1239
1242
1244
1247
1248
1249
1251
1252
1255
1256
1257
1258
1259
1260
1263
1264
1269
1271
1272
1278
1281
1284
1288
1289
1290
1291
1292
1294
1296
1301
1305
1306
1309
1310
1317
1318
1319
1320
1321
1322
1329
1330
1331
1336
1337
1339
1343
1344
1346
1350
1351
1352
1354
1355
1359
1360
1361
1365
1367
1370
1371
1372
1373
1374
1376
1379
1380
1381
1382
1385
1386
1388
1389
1393
1395
1399
1401
1402
1405
1406
1408
1409
1410
1411
1413
1414
1415
1417
1418
1419
1422
1423
1424
1429
1430
1434
1437
1438
1439
1440
1441
1442
1448
1449
1450
1452
1454
1462
1470
1472
1474
1476
1477
1478
1480
1484
1489
1490
1495
1496
1498
1499
1500
1503
1505
1506
1507
1510
1513
1518
1520
1521
1522
1523
1526
1528
1530
1532
1533
1538
1542
1544
1548
1

Exception: Request failed with status code 429.